In [19]:
!pip install langextract python-dotenv

In [20]:
import langextract as lx
import textwrap
import os
from dotenv import load_dotenv

In [21]:
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("LANGEXTRACT_API_KEY")

In [22]:
# Define the extraction task
prompt = textwrap.dedent("""\
Extract financial details in a structured way using the categories below:

1. Parties & Identification
- Capture names of companies, institutions, regulators, or counterparties.
- Record identifiers such as CIN, PAN, GST/VAT, Tax IDs, or any registration codes.
- Include account references like bank accounts, loan numbers, or investment account IDs.

2. Monetary Information
- Principal sums: invoice totals, loan values, transaction amounts.
- Charges & fees: late charges, service/processing fees, management costs.
- Interest terms: fixed/floating interest rates, APR values, or benchmark-linked references (e.g., SOFR, LIBOR).
- Taxes: GST, VAT, withholding, or similar levies.
- Penalties/Fines: early exit fee, defaults, or other financial penalties.

3. Dates & Timeframes
- Effective/Start dates: when agreements or transactions take effect.
- Maturity/Closing dates: final settlement or loan closure.
- Due dates: payment deadlines or installment schedules.
- Duration/tenure: repayment term, lock-in period, or ramp-up.
- Historical dates: transaction execution, invoice issue, settlement.

Instructions:
- Always extract **exact text spans** from the input (no rephrasing).
- Each extracted item must have contextual attributes (e.g., type of ID, kind of date, nature of monetary value).
""")


In [23]:
# Examples to guide extraction
examples = [
    lx.data.ExampleData(
        text="Tata Consultancy Services Limited (CIN: L99999MH1995PLC084781, PAN: AAACM8654F, GSTIN: 29AAACT1924F1Z9) issued Invoice No. TCS/INV/2024/987 on 15/09/2024 with a payment due date of 30/09/2024. The invoice carried IGST @ 18% on professional service fees. Payments must be credited to HDFC Bank, Account Number 123456789012, IFSC Code HDFC0001234. Any overdue payment will attract an additional 10% per annum as penalty interest.",
        extractions=[
            lx.data.Extraction(extraction_class="party", extraction_text="Tata Consultancy Services Limited", attributes={"type":"company"}),
            lx.data.Extraction(extraction_class="identifier", extraction_text="CIN: L99999MH1995PLC084781", attributes={"id_type":"CIN"}),
            lx.data.Extraction(extraction_class="identifier", extraction_text="PAN: AAACM8654F", attributes={"id_type":"PAN"}),
            lx.data.Extraction(extraction_class="identifier", extraction_text="GSTIN: 29AAACT1924F1Z9", attributes={"id_type":"GSTIN"}),
            lx.data.Extraction(extraction_class="party", extraction_text="HDFC Bank", attributes={"type":"bank"}),
            lx.data.Extraction(extraction_class="monetary", extraction_text="professional service fees", attributes={"value_type":"service_fee"}),
            lx.data.Extraction(extraction_class="interest_rate", extraction_text="10% per annum", attributes={"type":"penalty_interest"}),
            lx.data.Extraction(extraction_class="tax", extraction_text="IGST @ 18%", attributes={"tax_type":"GST"}),
            lx.data.Extraction(extraction_class="account", extraction_text="123456789012", attributes={"account_type":"bank"}),
            lx.data.Extraction(extraction_class="date", extraction_text="15/09/2024", attributes={"date_type":"invoice_date"}),
            lx.data.Extraction(extraction_class="date", extraction_text="30/09/2024", attributes={"date_type":"due_date"}),
        ]
    ),
]


In [24]:
# Input text
input_text = """
Infosys Limited (CIN: L85110KA1981PLC013115, PAN: AAACI4798L, GSTIN: 29AAACI4798L1ZU) issued Tax Invoice No. INF/INV/2025/204 on 15/02/2025 with a payment due date of 28/02/2025. The invoice included IGST @ 18% on the consulting charges. Payments are to be made to State Bank of India, Account Number 112233445566, IFSC Code SBIN0000456. If payment is delayed beyond the due date, a penalty interest of 10% per annum will apply.

Kotak Mahindra Bank issued its 8.20% Fixed Rate Senior Secured Bonds (ISIN: INE237A08765) on 12 January 2023. Each bond has a face value of ₹5,00,000 and carries a fixed coupon of 8.20% per annum, payable semi-annually. The maturity date is set for 12 January 2028.

On 20 March 2022, Standard Chartered India executed its first foreign trade finance deal linked to the SOFR benchmark, marking a shift from LIBOR-based transactions.

Punjab National Bank, under the SARFAESI Act, issued a notice on 10-09-2023 against borrower loan accounts LAC00987651234 and LAC00987659876. The secured property was taken into possession on 05-01-2024. Auction was scheduled with earnest money deposit due by 25-03-2024, and bids opening on 26-03-2024. As per Section 194-IA of the Income Tax Act, the buyer is liable to deduct TDS.

Bajaj Finance Limited launched Secured Redeemable Non-Convertible Debentures (NCDs) with a face value of ₹10,000 each. The issue had series with tenors of 24 months, 48 months, and 84 months. Coupon rates ranged between 8.50% and 9.40% per annum depending on the chosen series.
"""



In [25]:
# Run extraction
result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt,
    examples=examples,
    model_id="gemini-2.5-flash",
    extraction_passes=3,
    max_workers=20,
    max_char_buffer=1000,
    api_key=api_key
)


In [26]:
# Save results to JSONL
lx.io.save_annotated_documents([result], output_name="financial_data.jsonl", output_dir=".")

LangExtract: Saving to financial_data.jsonl: 1 docs [00:00, 205.58 docs/s]

✓ Saved 1 documents to financial_data.jsonl


In [27]:
# Generate interactive visualization
html_content = lx.visualize("financial_data.jsonl")

LangExtract: Loading financial_data.jsonl: 100%|██████████| 12.4k/12.4k [00:00<00:00, 12.1MB/s]

✓ Loaded 1 documents from financial_data.jsonl


In [28]:
# Save visualization to HTML file
with open("financial_data_visualization.html", "w") as f:
    if hasattr(html_content, 'data'):
        f.write(html_content.data)
    else:
        f.write(html_content)